In [41]:
import pandas as pd 
import numpy as np
from scipy.stats import ttest_ind
import scipy.stats as stats

#### Проведение A/B тестирования
##### Дана таблица: 

##### ***id_client*** - уникальный идентификатор клиента
##### **id_group*** - идентификатор группы (0 - контроль, 1 - тест)
##### ***city*** - название города
##### ***nflag_purchase*** - факт покупки (1 - приобретен рекламируемый товар, 0 - нет)
##### ***time_came*** - дата регистрации пользователя в приложении

##### Задача:
##### 1. Рассчитать результаты A/B Теста по всей совокупности и сделайте выводы.
##### 2. Рассчитать результаты A/B Теста для каждого города в отдельности и сделайте выводы.
##### 3. Рассчитать результаты A/B Теста для каждого города отдельно для тех, кто зарегистрировался в 2020 году, и отдельно для тех, кто зарегистрировался в 2021 году. Сделать выводы.

In [45]:
df = pd.read_csv('Домашняя работа Бизнес-3 (исходник).csv',sep= ';')
df.head(5)

,id_client,id_group,city,nflag_purchase,time_came
0,24360208,0,Moscow,1,21.10.2021
1,24360925,1,Moscow,1,16.10.2021
2,24357607,1,Saint-Petersbourg,0,09.01.2021
3,24362041,0,Saint-Petersbourg,0,25.01.2020
4,24360773,1,Novosibirsk,0,03.04.2021


#### Посмотрим на количество значений в тестовой и контрольной группе

In [48]:
df.groupby('id_group').agg(nflag_purchase_sum=('nflag_purchase', 'sum')).reset_index() 

,id_group,nflag_purchase_sum
0,0,872
1,1,974


#### Задача 1. Рассчитать АБ тест по всей совокупности

In [51]:
df['id_group'] = df['id_group'].astype(str)
s, p = ttest_ind(df[df['id_group'] == '0']['nflag_purchase'],df[df['id_group'] == '1']['nflag_purchase'])
print(s)
print(p)
print('средние не равны, H0 отклоняется на общей выборке')

-2.4013624185110922
0.01636252986305824
средние не равны, H0 отклоняется на общей выборке


#### Задача 2. Рассчитать АБ тест по каждому городу и по каждому году

In [54]:
for i in df['city'].unique():
    print(i)
    group_0 = df[(df['city'] == i) & (df['id_group'] == '0')]['nflag_purchase']
    group_1 = df[(df['city'] == i) & (df['id_group'] == '1')]['nflag_purchase']
    s, p = ttest_ind(group_0, group_1)
    print(p)    
    print()  
print('Средние равны в каждом городе кроме Владивостока')

Moscow
0.3981571415772427

Saint-Petersbourg
0.09701308927385824

Novosibirsk
0.6186980029170557

Kazan
0.592246594180279

Vladivostok
0.030257214768841512

Средние равны в каждом городе кроме Владивостока


#### Задача 3. Рассчитатьь АБ тест по каждому городу

In [57]:
# Преобразование столбца 'time_came' в datetime
df['time_came'] = pd.to_datetime(df['time_came'], errors='coerce')

for city in df['city'].unique():
    print(city)
    
    for year in [2020, 2021]:
        print(year)
        
        group_0 = df[(df['city'] == city) & (df['id_group'] == '0') & (df['time_came'].dt.year == year)]['nflag_purchase']
        group_1 = df[(df['city'] == city) & (df['id_group'] == '1') & (df['time_came'].dt.year == year)]['nflag_purchase']
        s, p = ttest_ind(group_0, group_1)
        print(p)

print('Вывод:')
print('Средние не равны, только во Владивостоке в 2020 году(p-value <, то есть только там АБ тест прошел удачно')

Moscow
2020
0.5054399863130574
2021
0.5753106595236979
Saint-Petersbourg
2020
0.28741355911452665
2021
0.19716903302230424
Novosibirsk
2020
0.8101519376946962
2021
0.6016373697833897
Kazan
2020
0.5426955440428094
2021
0.9122617463673748
Vladivostok
2020
0.003599171834113464
2021
0.6999017522609832
Вывод:
Средние не равны, только во Владивостоке в 2020 году(p-value <, то есть только там АБ тест прошел удачно


C:\Users\user\AppData\Local\Temp\ipykernel_10296\3630160546.py:2: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['time_came'] = pd.to_datetime(df['time_came'], errors='coerce')
